In [1]:
import numpy as np
import pandas as pd
import difflib#for comparing sequence
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies=pd.read_csv('movies.csv')
movies

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [3]:
movies.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movies.shape

(4803, 24)

In [5]:
selected_features=['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [6]:
for feature in selected_features:
    movies[feature]=movies[feature].fillna('')

In [8]:
combined_features=movies['genres']+' '+movies['keywords']+' '+movies['tagline']+' '+movies['cast']+' '+movies['director']
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [9]:
vec=TfidfVectorizer()

In [11]:
fea_vec=vec.fit_transform(combined_features)
print(fea_vec)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

In [14]:
similarity=cosine_similarity(fea_vec)
similarity

array([[1.        , 0.07219487, 0.037733  , ..., 0.        , 0.        ,
        0.        ],
       [0.07219487, 1.        , 0.03281499, ..., 0.03575545, 0.        ,
        0.        ],
       [0.037733  , 0.03281499, 1.        , ..., 0.        , 0.05389661,
        0.        ],
       ...,
       [0.        , 0.03575545, 0.        , ..., 1.        , 0.        ,
        0.02651502],
       [0.        , 0.        , 0.05389661, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02651502, 0.        ,
        1.        ]])

In [15]:
similarity.shape

(4803, 4803)

In [16]:
movie=input('Enter your favourite movie:')

Enter your favourite movie:Pirates of the Caribbean: At World's End


In [17]:
#Creating a list with all the movie name given in the dataset
list_of_all_titles=movies['title'].tolist()

In [19]:
find_close_match=difflib.get_close_matches(movie,list_of_all_titles)
find_close_match

["Pirates of the Caribbean: At World's End",
 "Pirates of the Caribbean: Dead Man's Chest",
 'Pirates of the Caribbean: On Stranger Tides']

In [23]:
close_match=find_close_match[0]
close_match

"Pirates of the Caribbean: At World's End"

In [24]:
index_of_the_movie=movies[movies.title==close_match]['index'].values[0]
index_of_the_movie

1

In [26]:
similarity_score=list(enumerate(similarity[index_of_the_movie]))
similarity_score

[(0, 0.07219486822992491),
 (1, 1.0000000000000004),
 (2, 0.032814990806793745),
 (3, 0.020741502866825225),
 (4, 0.03305463079248628),
 (5, 0.09894823392218867),
 (6, 0.014378790821108607),
 (7, 0.019781895260853298),
 (8, 0.04056180271185782),
 (9, 0.029948196927440142),
 (10, 0.06512426368922568),
 (11, 0.027580931446803184),
 (12, 0.5325423014890582),
 (13, 0.16315140691538466),
 (14, 0.05752279793612993),
 (15, 0.02384630652060951),
 (16, 0.018757441743698592),
 (17, 0.08693929581992486),
 (18, 0.017747962646584),
 (19, 0.061598207762639565),
 (20, 0.08780812424608472),
 (21, 0.016447740747051347),
 (22, 0.0473755607656848),
 (23, 0.044397771855870655),
 (24, 0.07038585257947678),
 (25, 0.0),
 (26, 0.02041663626765186),
 (27, 0.1458486783441737),
 (28, 0.06070748232215177),
 (29, 0.020505067521231088),
 (30, 0.09738646725014082),
 (31, 0.045173032380404654),
 (32, 0.08732451789987261),
 (33, 0.019658984415172717),
 (34, 0.0),
 (35, 0.020637015141285384),
 (36, 0.018595597513519024

In [28]:
sorted_similar_movies=sorted(similarity_score,key=lambda x:x[1],reverse=True)
sorted_similar_movies

[(1, 1.0000000000000004),
 (199, 0.5466481253706559),
 (12, 0.5325423014890582),
 (536, 0.288818238643064),
 (1652, 0.19164772894913185),
 (384, 0.18874711394669408),
 (13, 0.16315140691538466),
 (129, 0.15873396251727565),
 (3055, 0.15577364618957568),
 (3033, 0.15459290728645847),
 (329, 0.15057603842736808),
 (178, 0.14725915063827233),
 (27, 0.1458486783441737),
 (2035, 0.14149462529605056),
 (267, 0.14000715404465205),
 (1183, 0.13987145541658172),
 (326, 0.13846182473796617),
 (1095, 0.13674871062299218),
 (1890, 0.13658339038642311),
 (2401, 0.13627388719755615),
 (2040, 0.13546842154573419),
 (2474, 0.13515170184139358),
 (2254, 0.13504511148016585),
 (1577, 0.12992151193805673),
 (894, 0.12882247930032706),
 (1061, 0.1285840875411398),
 (189, 0.12748140950217185),
 (2515, 0.1263816681891445),
 (2815, 0.12602945523943587),
 (1063, 0.125934979358607),
 (1635, 0.12565457498567212),
 (840, 0.12427647655345161),
 (262, 0.12268102046117293),
 (128, 0.12130199878288984),
 (517, 0.120

In [29]:
print('Movies suggested for you:\n')
i=1
for movie in sorted_similar_movies:
    index=movie[0]
    title_from_index=movies[movies.index==index]['title'].values[0]
    if(i<30):
        print(i,'.',title_from_index)
    i+=1

Movies suggested for you:

1 . Pirates of the Caribbean: At World's End
2 . Pirates of the Caribbean: The Curse of the Black Pearl
3 . Pirates of the Caribbean: Dead Man's Chest
4 . Anna and the King
5 . Dragonball Evolution
6 . Cast Away
7 . The Lone Ranger
8 . Thor
9 . Seeking a Friend for the End of the World
10 . The Corruptor
11 . The Lord of the Rings: The Return of the King
12 . Rango
13 . Battleship
14 . Our Kind of Traitor
15 . Kingdom of Heaven
16 . The Mexican
17 . Cinderella
18 . Curse of the Golden Flower
19 . Don Juan DeMarco
20 . City of Ghosts
21 . The Glass House
22 . Arn: The Knight Templar
23 . The Children of Huang Shi
24 . Without a Paddle
25 . Bulletproof Monk
26 . Love Actually
27 . Noah
28 . Crouching Tiger, Hidden Dragon
29 . Star Trek II: The Wrath of Khan


In [31]:
movie=input('Enter your favourite movie:')
list_of_all_titles=movies['title'].tolist()
find_close_match=difflib.get_close_matches(movie,list_of_all_titles)
close_match=find_close_match[0]
index_of_the_movie=movies[movies.title==close_match]['index'].values[0]
similarity_score=list(enumerate(similarity[index_of_the_movie]))
sorted_similar_movies=sorted(similarity_score,key=lambda x:x[1],reverse=True)

print('Movies suggested for you:\n')
i=1
for movie in sorted_similar_movies:
    index=movie[0]
    title_from_index=movies[movies.index==index]['title'].values[0]
    if(i<30):
        print(i,'.',title_from_index)
    i+=1

Enter your favourite movie:Thor
Movies suggested for you:

1 . Thor
2 . Thor: The Dark World
3 . The Avengers
4 . Cinderella
5 . Avengers: Age of Ultron
6 . Jack Ryan: Shadow Recruit
7 . Captain America: The Winter Soldier
8 . Hamlet
9 . Henry V
10 . Much Ado About Nothing
11 . Pirates of the Caribbean: Dead Man's Chest
12 . Angels & Demons
13 . Pirates of the Caribbean: At World's End
14 . Captain America: Civil War
15 . City of Ghosts
16 . Our Kind of Traitor
17 . How to Train Your Dragon
18 . Spider-Man 2
19 . Ronin
20 . Bad Company
21 . Deadpool
22 . The Amazing Spider-Man 2
23 . Aberdeen
24 . Iron Man 2
25 . The Girl with the Dragon Tattoo
26 . Warcraft
27 . The Glass House
28 . X-Men: Days of Future Past
29 . Exorcist: The Beginning
